# OpenStreetMap - get crossing features

In [ ]:
# Import dependencies
import osmnx as ox
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import folium
import functions as f
import config_azure as cf
import geopandas as gpd
import pandas as pd

In [ ]:
os.system('sudo blobfuse /home/azureuser/cloudfiles/code/blobfuse/sidewalk --tmp-path=/mnt/resource/blobfusetmp --config-file=/home/azureuser/cloudfiles/code/blobfuse/fuse_connection_sidewalk.cfg -o attr_timeout=3600 -o entry_timeout=3600 -o negative_timeout=3600 -o allow_other -o nonempty')

### Get mask of area

In [ ]:
# Import areas
CRS = 'EPSG:4326'
df_areas = gpd.read_file(cf.output_pilot_area)
polygon = df_areas.to_crs(crs=CRS).unary_union

### Retrieve crossing features

In [ ]:
# Retrieve crossing attributes
tags = {"crossing": True, "footway": 'crossing', "kerb": True}
crossing_features = ox.features_from_polygon(polygon, tags)    

### Generate map and export crossing features

In [ ]:
# Create tooltip for feature representation on map
def gen_tooltip(fields, aliases):

    tooltip = folium.GeoJsonTooltip(
        fields=fields,
        aliases=aliases,
        localize=True,
        sticky=False,
        labels=True,
        style="""
            background-color: #F0EFEF;
            border: 2px solid black;
            border-radius: 3px;
            box-shadow: 3px;
        """,
        max_width=800,
    )
    return tooltip

# set True for satelite background, False for regular background
satelite = False

# Set folium map background
if satelite == True:
    network_color = 'white'
    tile = folium.TileLayer(
                tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
                attr = 'Esri',
                name = 'Esri Satellite',
                overlay = False,
                control = True)
else:
    tile = 'openstreetmap'
    network_color = 'black'

# Create Folium map
map = folium.Map(
    location=[52.389164, 4.908453], tiles=tile,
    min_zoom=10, max_zoom=25, zoom_start=15,
    zoom_control=True, control_scale=True, control=False
    )

# Add crossing features
feature_names = crossing_features.columns.tolist()
feature_names.remove('geometry')
tooltip = gen_tooltip(feature_names, feature_names)
geo_j = folium.GeoJson(crossing_features, tooltip=tooltip, style_function=lambda x: {"color": "orange", "weight": 10}).add_to(map)

map

In [ ]:
# Save crossing features as csv
crossing_features.to_csv(cf.out_folder + '/crossing_features/OpenStreetMap/crossing_features_osm.csv')

# Save network with crossing features as html map
if satelite == True:
    map.save(cf.out_folder + '/crossing_features/OpenStreetMap/crossing_features_osm_satelite.html')
else:
    map.save(cf.out_folder + '/crossing_features/OpenStreetMap/crossing_features_osm.html')